In [1]:
import os
import glob

from benthic_mapping.download_media import MediaDownloader
from benthic_mapping.fiftyone_clustering import FiftyOneDatasetViewer

In [2]:
# Initialize the downloader with the required parameters
downloader = MediaDownloader(
    api_token=os.getenv("TATOR_TOKEN"),
    project_id=155,
    output_dir="../Data"
)

# Download the media
media_ids = ["14759824"]
downloader.download_data(media_ids, 
                         convert=False, 
                         extract=True, 
                         every_n_seconds=60)

NOTE: Authentication successful for jordan.pierce
NOTE: Media GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH.mp4 downloaded successfully to ../Data\Raw_Videos\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4
NOTE: Extracting frames from GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH.mp4...
Video duration: 381.88 seconds
Extracting 7 frames using 7 workers...
Successfully extracted 7 frames to ../Data\Extracted_Frames\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted


['../Data\\Raw_Videos\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4']

In [7]:
downloader.media_path_map

{'14759824': {'original': '../Data\\Raw_Videos\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted.mp4',
  'converted': None,
  'frames': '../Data\\Extracted_Frames\\GL2301_VID_20230725T145731Z_D015_DROPCAM_HIGH_converted'}}